# Digits Generation with GAN
## Import Packages

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import time

## Utilities

In [ ]:
def sample_images(images, row_count, column_count):
    fig, axs = plt.subplots(row_count, column_count, figsize=(10,10))
    for i in range(row_count):
        for j in range(column_count):
            axs[i,j].imshow(images[i * column_count + j])
            axs[i,j].axis('off')
    plt.show()

In [ ]:
def genrate_images(generator,row_count, column_count):
    fake_images = generator(tf.random.normal([row_count * column_count, random_normal_dimensions]))
    sample_images(fake_images, row_count, column_count)

## Import Datasets

In [ ]:
batch_size = 100
random_normal_dimensions = 32
n_epochs = 10

In [ ]:
def preproces_image(item):
    image = item["image"]
    image = tf.cast(image, "float")  / 255.0
    image =tf.reshape(image, (28, 28))
    return image

In [ ]:
dataset = tfds.load("mnist", split='train', as_supervised=False).map(preproces_image).shuffle(1024).batch(batch_size, drop_remainder=True).prefetch(1).repeat(n_epochs)

## Build the Generator

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
generator = keras.models.Sequential([                                 
    keras.layers.Dense(64, activation="selu", input_shape=[random_normal_dimensions]),
    keras.layers.Dense(128, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

In [ ]:
generator.summary()

## Build the Discriminator

In [ ]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(128, activation="selu"),
    keras.layers.Dense(64, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
discriminator.summary()

## Build the GAN

In [ ]:
gan = keras.Sequential([generator, discriminator])

In [ ]:
gan.summary()

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

## Train the Model

In [ ]:
begin = time.time()
current_traning_images = 0
total = 60000 * n_epochs
i = 0
for real_images in dataset:
    noise = tf.random.normal(shape=[batch_size, random_normal_dimensions])
    fake_images = generator(noise)
    current_traning_images += fake_images.shape[0]
    mixed_images = tf.concat([fake_images, real_images], axis=0)
    discriminator_labels = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
    discriminator.trainable = True
    discriminator.train_on_batch(mixed_images, discriminator_labels)
    noise = tf.random.normal(shape=[batch_size, random_normal_dimensions])
    generator_labels = tf.constant([[1.]] * batch_size)
    discriminator.trainable = False
    gan.train_on_batch(noise, generator_labels)
    if i > 0 and i % 500 == 0:
        current_time = time.time() - begin
        ETA = current_time / current_traning_images * total - current_time
        print("ETA: %.2fs"% (ETA))
        genrate_images(generator,10, 10)
    i += 1

In [ ]:
genrate_images(generator,10, 10)

## Evaluation

In [ ]:
sample_count = 512
noise = tf.random.normal(shape=[sample_count, random_normal_dimensions])
fake_images = generator(noise)
probs = discriminator.predict(fake_images)
y_true = np.array([0.0] * sample_count)
y_pred = np.array(probs > 0.5, dtype=int)

### BCE

In [ ]:
bce = tf.keras.metrics.BinaryCrossentropy()(y_true, y_pred)
print("BCE:%.2f"%(bce))

## Accuracy

This means that there is 82% of chance that Discriminator can classify the images generated by Generator fake.

In [ ]:
accuracy = tf.keras.metrics.Accuracy()(y_true, y_pred)
print("Accuracy:%.2f"%(accuracy))

## Save the Model

In [ ]:
generator.save("generator.h5")

In [ ]:
discriminator.save("discriminator.h5")

In [ ]:
gan.save("gan.h5")